In [1]:
import numpy as np

np.random.seed(0)

NUM_EXAMPLES = 1000
NUM_FEATURES = 20

CLASS0_PROBS = np.random.uniform(size = NUM_FEATURES)
CLASS0_ODDS = CLASS0_PROBS / (1 - CLASS0_PROBS)
LOG_ODDS_RATIOS = 2 * np.random.uniform(low = -1, high = +1, size = NUM_FEATURES)
CLASS1_ODDS = CLASS0_ODDS * np.exp(LOG_ODDS_RATIOS)
CLASS1_PROBS = CLASS1_ODDS / (1 + CLASS1_ODDS)

training_x = []
training_y = []
for _ in range(NUM_EXAMPLES):
    y = np.random.uniform() > 0.50
    
    if y == 1:
        x = np.random.binomial(1, CLASS1_PROBS, NUM_FEATURES)
    else:
        x = np.random.binomial(1, CLASS0_PROBS, NUM_FEATURES)
    training_x.append(x)
    training_y.append(y)


In [2]:
training_x[:10]

[array([1, 1, 1, 0, 1, 1, 0, 1, 1, 0, 1, 0, 1, 1, 0, 0, 0, 1, 1, 1]),
 array([0, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 1]),
 array([0, 1, 1, 1, 0, 1, 1, 1, 1, 0, 1, 1, 1, 1, 0, 0, 0, 1, 1, 1]),
 array([1, 1, 0, 1, 0, 0, 0, 1, 1, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 1]),
 array([0, 1, 1, 0, 0, 1, 0, 1, 1, 0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 1]),
 array([1, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 1, 1, 1, 0, 0, 0, 1, 0, 1]),
 array([1, 0, 1, 1, 0, 1, 0, 1, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 1, 1]),
 array([1, 1, 1, 1, 0, 0, 0, 1, 1, 0, 1, 1, 1, 1, 0, 0, 0, 1, 1, 1]),
 array([0, 0, 1, 1, 0, 1, 0, 1, 1, 0, 0, 1, 0, 1, 0, 0, 0, 1, 1, 0]),
 array([1, 1, 1, 1, 0, 1, 0, 1, 1, 0, 0, 1, 1, 1, 0, 0, 0, 1, 1, 1])]

In [3]:
# How many examples are there?
# How many features are there per example?
# How many present features does the average example have?
# * Loop through, summing for each example, and then average.
# How many positive and negative examples are there?

print(len(training_x))
print(len(training_x[0]))
print(
    sum([sum(x) for x in training_x]) / len(training_x)
)
print(sum(training_y))
print(len(training_y) - sum(training_y))

1000
20
11.893
486
514


In [4]:
IDEAL_BIAS = 0 + np.sum(np.log((1 - CLASS1_PROBS) / (1 - CLASS0_PROBS)))
IDEAL_THETAS = np.log(
    (CLASS1_PROBS / CLASS0_PROBS)
    /
    ((1 - CLASS1_PROBS) / (1 - CLASS0_PROBS))
)

# Use the ideal theta to sum up a score for every example. Give me this as an array called "z_scores"
# Define a function called z_score(example, theta)

def z_score(example, thetas, bias):
    z = bias
    for i in range(NUM_FEATURES):
        z += example[i] * thetas[i]
    return z

z_scores = [
    z_score(example, IDEAL_THETAS, IDEAL_BIAS) for example in training_x
]

# Print out the first ten z scores.
print(z_scores[:10])

[-1.178747493114189, -2.3679080715482272, -0.7738539530021048, 1.5037886406277452, -3.1659122084456701, -1.3317132325566838, 0.74767881577146877, 2.0075221809826713, -0.15400633320381951, 3.5089838178742587]


In [5]:
# Write a classify function that takes in the set of x values and outputs the predictions based on z>0 or not.
# Write a function that takes in a list of predictions and true ys and calculates the error.

def classify(examples, thetas, bias):
    predictions = []
    for idx, example in enumerate(examples):
        z = z_score(example, thetas, bias)
        if z > 0.0:
            predictions.append(1)
        else:
            predictions.append(0)
    return predictions

def accuracy(predictions, targets):
    num_correct = sum([
        1 if predictions[idx] == targets[idx]
        else 0
        for idx in range(len(predictions))
    ])
    
    return num_correct / len(predictions)

accuracy(
    classify(training_x, IDEAL_THETAS, IDEAL_BIAS),
    training_y
)

0.819

In [6]:
# Let's see a way to turn the z values into probabilities.
# We'll use (e^z / (1 + e^z)). This is called the logistic function.

def logistic(z):
    return (np.exp(z) / (1 + np.exp(z)))

# Calculate the probabilities for every example.

example_probs = list(map(logistic, z_scores))

# Write a function classify2 that classifies as positive if prob > 0.50.
# Calculate the accuracy error.
# Compare to the result of the original classify. Why are they the same?

def classify2(probabilities):
    return [1 if prob > 0.50 else 0 for prob in probabilities]

accuracy(
    classify2(example_probs),
    training_y
)

0.819

In [7]:
# Define a function called mean_absolute_error. This takes in
# a list of probabilities and of target values.
# Sum the absolute difference and average.

def mean_absolute_error(probabilities, targets):
    return sum([
        np.abs(targets[idx] - probabilities[idx])
        for idx in range(len(probabilities))
    ]) / len(probabilities)

mean_absolute_error(example_probs, training_y)

0.25854523218263092

In [8]:
# Calculate the z_scores for if you had only zero theta values.
# What values do you get? Why?
# Calculate the probabilities. What are they? Why.
# Calculate the predictions. Use either classify or classify2. What do you get? Why?
# Calculate the accuracy. What do you get. Why?
# Calculate the mean_absolute_error. What is this? Why?

def zero_thetas():
    return [0 for _ in range(NUM_FEATURES)]
ZERO_BIAS = 0.0

z_scores = [
    z_score(example, zero_thetas(), ZERO_BIAS)
    for
    example in training_x
]
print(z_scores[:10])
probs = list(map(logistic, z_scores))
print(probs[:10])
predictions = classify2(probs)
print(accuracy(predictions, training_y))
print(mean_absolute_error(predictions, training_y))

[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
[0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5]
0.514
0.486


In [9]:
# Write a function called compare_thetas. It should take in the examples,
# plus an old set of thetas, and a new one. Compare the errors on one versus the other.
# Return the difference in errors.

def compare_thetas(examples, training_y, old_thetas, old_bias, new_thetas, new_bias):
    old_probs = [
        logistic(z_score(example, old_thetas, old_bias))
        for
        example in examples
    ]
    new_probs = [
        logistic(z_score(example, new_thetas, new_bias))
        for
        example in examples
    ]
    
    old_error = mean_absolute_error(old_probs, training_y)
    new_error = mean_absolute_error(new_probs, training_y)
    return new_error - old_error

# Compare the ZERO_THETAS and the IDEAL_THETAS.
# What sign represents improvement?
compare_thetas(training_x, training_y, IDEAL_THETAS, IDEAL_BIAS, zero_thetas(), ZERO_BIAS)

0.24145476781736908

In [10]:
# Write a function called `tweak_theta_at_idx`.
# It tries to tweak a parameter up or down, to see which one yields improvement.
# If neither does, it leaves it.
# Do not modify the original thetas array! Return a new tweaked thetas array.

def tweak_theta_at_idx(examples, training_y, old_thetas, old_bias, idx, step_size):
    new_thetas = list(old_thetas)
    new_thetas[idx] += step_size
    if compare_thetas(examples, training_y, old_thetas, old_bias, new_thetas, old_bias) < 0.0:
        return new_thetas

    new_thetas = list(old_thetas)
    new_thetas[idx] -= step_size
    if compare_thetas(examples, training_y, old_thetas, old_bias, new_thetas, old_bias) < 0.0:
        return new_thetas

    return old_thetas

def tweak_bias(examples, training_y, old_thetas, old_bias, step_size):
    new_bias = old_bias + step_size
    if compare_thetas(examples, training_y, old_thetas, old_bias, old_thetas, new_bias) < 0.0:
        return new_bias

    new_bias = old_bias - step_size
    if compare_thetas(examples, training_y, old_thetas, old_bias, old_thetas, new_bias) < 0.0:
        return new_bias

    return new_bias

# Write a second function called tweak_all_thetas.
def tweak_all_thetas(examples, training_y, thetas, bias, step_size):
    for idx in range(len(thetas)):
        thetas = tweak_theta_at_idx(examples, training_y, thetas, bias, idx, step_size)
    bias = tweak_bias(examples, training_y, thetas, bias, step_size)
    return (thetas, bias)

# Now apply it to your ZERO_THETAS with a step size of 0.10. Print out what you get.
new_thetas, new_bias = tweak_all_thetas(training_x, training_y, zero_thetas(), ZERO_BIAS, 0.10)


In [11]:
print(new_thetas)
print(
    mean_absolute_error(
        [logistic(z_score(x, new_thetas, new_bias)) for x in training_x],
        training_y
    )
)
print(
    mean_absolute_error(
        [logistic(z_score(x, zero_thetas(), ZERO_BIAS)) for x in training_x],
        training_y
    )
)


[0.1, 0.1, -0.1, 0.1, -0.1, 0.1, -0.1, 0.1, -0.1, -0.1, -0.1, 0.1, -0.1, -0.1, -0.1, 0.1, 0.1, 0.1, 0.1, -0.1]
0.469144257827
0.5


In [12]:
# Write a loop to tweak the thetas 10 times. Print out the MAE each time.

thetas = zero_thetas()
bias = ZERO_BIAS
for _ in range(10):
    thetas, bias = tweak_all_thetas(training_x, training_y, thetas, bias, 0.10)

    print(
        mean_absolute_error(
            [logistic(z_score(x, thetas, bias)) for x in training_x],
            training_y
        )
    )


0.469144257827
0.439799326946
0.412700986874
0.389063143023
0.368264503576
0.350378609247
0.334182431195
0.320983105611
0.309076373717
0.298400956877


In [13]:
# Print out how well your thetas do for classification.
accuracy(
    classify(
        training_x,
        thetas,
        bias
    ),
    training_y
)


0.798

In [14]:
# Currently in order to tweak a theta, we much go through the
# entire dataset to evaluate whether the change is good.
# If we have twenty parameters, we must do this twenty times
# before we have updated all the parameters.
# In fact, we must do this *more* than once, in the case that
# an increase is bad, we need to check about a decrease.
# This problem will be worse with more parameters.
#
# Let's improve things by looking at a *single* example and
# asking it to "vote" whether it is better to increase a
# parameter or decrease it.
#
# If the example is a positive example, it wants to increase
# the z value. If it is a negative example, it wants to
# decrease the z value.
#
# If a feature is present, it will want to decrease or increase
# the corresponding parameter. But if the feature is absent, it
# has no vote for this parameter.
#
# Votes should be -1, 0, or +1.

def calculate_example_votes(example, label):
    vote_direction = (2 * label) - 1
    votes = [
        vote_direction * x
        for x in example
    ]
    bias_vote = vote_direction
    
    return votes, bias_vote

# Calculate and print the first ten votes:
for idx in range(10):
    print(
        calculate_example_votes(training_x[idx], training_y[idx])
    )

([-1, -1, -1, 0, -1, -1, 0, -1, -1, 0, -1, 0, -1, -1, 0, 0, 0, -1, -1, -1], -1)
([0, -1, -1, -1, -1, -1, -1, -1, 0, 0, 0, 0, 0, -1, 0, 0, 0, -1, -1, -1], -1)
([0, -1, -1, -1, 0, -1, -1, -1, -1, 0, -1, -1, -1, -1, 0, 0, 0, -1, -1, -1], -1)
([1, 1, 0, 1, 0, 0, 0, 1, 1, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 1], 1)
([0, -1, -1, 0, 0, -1, 0, -1, -1, 0, -1, 0, 0, -1, 0, 0, 0, -1, 0, -1], -1)
([1, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 1, 1, 1, 0, 0, 0, 1, 0, 1], 1)
([-1, 0, -1, -1, 0, -1, 0, -1, -1, 0, 0, 0, -1, -1, 0, 0, 0, 0, -1, -1], -1)
([1, 1, 1, 1, 0, 0, 0, 1, 1, 0, 1, 1, 1, 1, 0, 0, 0, 1, 1, 1], 1)
([0, 0, 1, 1, 0, 1, 0, 1, 1, 0, 0, 1, 0, 1, 0, 0, 0, 1, 1, 0], 1)
([1, 1, 1, 1, 0, 1, 0, 1, 1, 0, 0, 1, 1, 1, 0, 0, 0, 1, 1, 1], 1)


In [15]:
# Write a function that calculates the "average" vote.
#
# To keep track of sums of arrays, it is nice to use
# numpy, which can vectorize additions.
def calculate_all_votes(training_x, training_y):
    total_votes = np.zeros(NUM_FEATURES)
    total_bias_votes = 0.0
    for (x, y) in zip(training_x, training_y):
        votes, bias_vote = calculate_example_votes(x, y)
        total_votes += np.array(votes)
        total_bias_votes += bias_vote
    return (
        total_votes / len(training_x),
        total_bias_votes / len(training_x)
    )

calculate_all_votes(training_x, training_y)

(array([ 0.126,  0.085, -0.048,  0.126, -0.14 ,  0.032, -0.136,  0.032,
        -0.017, -0.051, -0.137,  0.107, -0.03 , -0.026, -0.03 ,  0.005,
         0.007,  0.023,  0.062,  0.003]), -0.028)

In [16]:
# Write a loop to ten times make the average voted update, printing the MAE each time.

thetas = zero_thetas()
bias = ZERO_BIAS
for _ in range(10):
    votes, bias_vote = calculate_all_votes(training_x, training_y)
    thetas = [
        theta + vote for (theta, vote) in zip(thetas, votes)
    ]
    bias += bias_vote

    print(
        mean_absolute_error(
            [logistic(z_score(x, thetas, bias)) for x in training_x],
            training_y
        )
    )
    # Print the classification accuracy. This should be significantly lower than before.
    print(
        accuracy(
            classify(
                training_x,
                thetas,
                bias
            ),
            training_y
        )
    )

# Print the thetas. How are they related to the original thetas?
print(thetas)

# Try to run 100 iterations to see if your accuracy can improve the accuracy...
# (You probably cannot)


0.46974722225
0.709
0.441238416745
0.709
0.415735849892
0.709
0.393827498095
0.709
0.375532830366
0.708
0.360528449487
0.709
0.34834697224
0.709
0.338502198417
0.709
0.330551137579
0.709
0.324116646217
0.709
[1.2599999999999998, 0.84999999999999987, -0.47999999999999993, 1.2599999999999998, -1.4000000000000004, 0.32000000000000006, -1.3600000000000003, 0.32000000000000006, -0.17000000000000004, -0.51000000000000001, -1.3700000000000001, 1.0700000000000001, -0.30000000000000004, -0.26000000000000001, -0.30000000000000004, 0.049999999999999996, 0.070000000000000007, 0.22999999999999995, 0.62000000000000011, 0.029999999999999995]


In [27]:
# The problem is that we add the same vote all the time. Each example should vote
# based on "need." Examples that are well classified should vote less.
# Or the flip side is that examples that can improve a lot from a change should vote more.

def calculate_vote_weight(example, label, thetas, bias):
    p = logistic(z_score(example, thetas, bias))
    vote_weight = 2 * np.abs(label - p)
    return vote_weight

def calculate_all_votes2(training_x, training_y, thetas, bias):
    total_votes = np.zeros(NUM_FEATURES)
    total_bias_votes = 0.0
    for (x, y) in zip(training_x, training_y):
        votes, bias_vote = calculate_example_votes(x, y)
        vote_weight = calculate_vote_weight(x, y, thetas, bias)
        total_votes += np.array(votes) * vote_weight
        total_bias_votes += bias_vote * vote_weight

    return (
        total_votes / len(training_x),
        total_bias_votes / len(training_x)
    )

calculate_all_votes2(training_x, training_y, zero_thetas(), ZERO_BIAS)


(array([ 0.126,  0.085, -0.048,  0.126, -0.14 ,  0.032, -0.136,  0.032,
        -0.017, -0.051, -0.137,  0.107, -0.03 , -0.026, -0.03 ,  0.005,
         0.007,  0.023,  0.062,  0.003]), -0.028000000000000001)

In [33]:
# Write a loop to ten times make the average voted update, printing the MAE each time.

thetas = zero_thetas()
bias = ZERO_BIAS

LEARNING_RATE = 0.2

for _ in range(100):
    votes, bias_vote = calculate_all_votes2(training_x, training_y, thetas, bias)
    thetas = [
        theta + vote * LEARNING_RATE for (theta, vote) in zip(thetas, votes)
    ]
    bias += bias_vote * LEARNING_RATE

    print("MAE")
    print(
        mean_absolute_error(
            [logistic(z_score(x, thetas, bias)) for x in training_x],
            training_y
        )
    )
    # Print the classification accuracy. This should be significantly lower than before.
    print("ACCURACY")
    print(
        accuracy(
            classify(
                training_x,
                thetas,
                bias
            ),
            training_y
        )
    )

# Print the thetas. How are they related to the original thetas?
print(thetas)

# Try to run 100 iterations to see if your accuracy can improve the accuracy...
# (You probably cannot)


MAE
0.493890064661
ACCURACY
0.708
MAE
0.488072699406
ACCURACY
0.759
MAE
0.482448056314
ACCURACY
0.776
MAE
0.477015678041
ACCURACY
0.778
MAE
0.471773796379
ACCURACY
0.783
MAE
0.466719529663
ACCURACY
0.788
MAE
0.461849089341
ACCURACY
0.79
MAE
0.457157967869
ACCURACY
0.788
MAE
0.45264110609
ACCURACY
0.789
MAE
0.448293039603
ACCURACY
0.789
MAE
0.444108024597
ACCURACY
0.786
MAE
0.44008014436
ACCURACY
0.788
MAE
0.436203398028
ACCURACY
0.788
MAE
0.432471773414
ACCURACY
0.791
MAE
0.428879305799
ACCURACY
0.791
MAE
0.425420124539
ACCURACY
0.79
MAE
0.422088489253
ACCURACY
0.79
MAE
0.418878817184
ACCURACY
0.791
MAE
0.41578570319
ACCURACY
0.791
MAE
0.412803933635
ACCURACY
0.794
MAE
0.409928495283
ACCURACY
0.794
MAE
0.407154580153
ACCURACY
0.794
MAE
0.404477587137
ACCURACY
0.795
MAE
0.401893121061
ACCURACY
0.795
MAE
0.399396989759
ACCURACY
0.795
MAE
0.396985199632
ACCURACY
0.795
MAE
0.394653950072
ACCURACY
0.795
MAE
0.392399627072
ACCURACY
0.795
MAE
0.390218796278
ACCURACY
0.797
MAE
0.388108195686
A

KeyboardInterrupt: 